In [87]:
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
import numpy as np
from fiona.drvsupport import supported_drivers
from simplekml import Kml, Color
supported_drivers['KML'] = 'rw'
supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled 
import numpy as np
import geopandas as gpd
from simplekml import Kml, Color
import os
import time
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from shapely.ops import unary_union

In [5]:
def generate_bins(geom, bin_size=10):

    x_ref, y_ref = 193602, 2602072
    xmin, ymin, xmax, ymax = geom.bounds
    # To shift the polygon to math the whole grid
    xmin = math.floor((xmin-x_ref)/bin_size)*bin_size+x_ref
    xmax = math.ceil((xmax-x_ref)/bin_size)*bin_size+x_ref
    ymin = math.floor((ymin-y_ref)/bin_size)*bin_size+y_ref
    ymax = math.ceil((ymax-y_ref)/bin_size)*bin_size+y_ref
    # Calculate the number of bins in each dimension
    xbins = int((xmax - xmin) / bin_size)
    ybins = int((ymax - ymin) / bin_size)
    # Create arrays of x and y coordinates for bins
    x_coords = np.arange(xmin, xmin + xbins * bin_size, bin_size)
    y_coords = np.arange(ymin, ymin + ybins * bin_size, bin_size)
    # Use NumPy's meshgrid to generate all combinations of x and y coordinates
    x_grid, y_grid = np.meshgrid(x_coords, y_coords)
    # Flatten the arrays and create bin polygons using vectorized operations
    x_flat = x_grid.flatten()
    y_flat = y_grid.flatten()
    bin_polygons = [
        Polygon([
            (x, y), (x + bin_size, y), (x + bin_size, y + bin_size), (x, y + bin_size)
        ]) for x, y in zip(x_flat, y_flat)
    ]
    return bin_polygons

In [74]:
## Get border file and raw data file
# area_name = "DN_KCN_AN_DON"
area_name = "DN_KCN_HOA_KHANH"
# area_name = "QN_KCN_VSIP"
# area_name = "KH_QL1"
date_str = '210723'
bin_size = 10

input_polygon_file = f"../Polygon/{area_name}.kml"
polygon_df = gpd.read_file(input_polygon_file)
          
# input_raw_file = f"../Raw/{area_name}/{area_name}_{date_str}.csv"
# df_raw = pd.read_csv(input_raw_file)

folder_path = f"../Raw/{area_name}/Pilot_{date_str}"
print(f"Reading from multiple input raw files in {folder_path}")
df_list = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, low_memory=False)
        df_list.append(df)
df_raw = pd.concat(df_list, ignore_index=True)

Reading from multiple input raw files in ../Raw/DN_KCN_HOA_KHANH/Pilot_210723


In [75]:
## Divide bins and locate points
print(f"***===RUNNING:{area_name}, {bin_size}*{bin_size} grid ==***")
### Read a Polygon and divide
polygon_df = polygon_df.to_crs('EPSG:32648')
bins_polygons = [generate_bins(geom, bin_size) for geom in polygon_df['geometry']]
bins_polygons = [bin_poly for sublist in bins_polygons for bin_poly in sublist]
# Create the GeoDataFrame for bins
bins_df = gpd.GeoDataFrame({'geometry': bins_polygons}, crs=polygon_df.crs)
bins_df['centroid'] = bins_df['geometry'].centroid
bins_df_4326 = bins_df.to_crs("EPSG:4326")
bins_df_4326['centroid'] = bins_df_4326['centroid'].to_crs("EPSG:4326")
bins_df_4326['longitude'] = bins_df_4326['centroid'].x
bins_df_4326['latitude'] = bins_df_4326['centroid'].y
bins_df_4326['bin_id'] = "bin" + "_" +bins_df_4326['latitude'].astype(str)+ "_" + bins_df_4326['longitude'].astype(str)
bins_df_4326 = bins_df_4326[['bin_id', 'geometry']]
print('==Complete dividing bins')
### Load raw data file
df_point = df_raw.copy()
df_point_columns = [
    'Start Time',
    'eNodeB',
    'EARFCN (DL)',
    'EARFCN (UL)',
    'Physical Cell ID',
    'Latitude',
    'Longitude',
    'UL Volume (kB)',
    'DL Volume (kB)',
    'CQI 0',
    'CQI 1',
    'CQI 2',
    'CQI 3',
    'CQI 4',
    'CQI 5',
    'CQI 6',
    'CQI 7',
    'CQI 8',
    'CQI 9',
    'CQI 10',
    'CQI 11',
    'CQI 12',
    'CQI 13',
    'CQI 14',
    'CQI 15',
    'Serving Cell Label',
    'Serving Cell RSRP',
    'Best Cell Label',
    'Best Cell RSRP',
    'Second Best Cell Label',
    'Second Best Cell RSRP',
    'Third Best Cell Label',
    'Third Best Cell RSRP',
    'Fourth Best Cell Label',
    'Fourth Best Cell RSRP',
    'Fifth Best Cell Label',
    'Fifth Best Cell RSRP',
    'Sixth Best Cell Label',
    'Sixth Best Cell RSRP',
]
df_point = df_point[df_point_columns]

### Locate bins for each sample
bins_df_4326.reset_index(inplace=True)
polygonDF = bins_df_4326.copy()
polygonDF = polygonDF[['bin_id','geometry']]
# Create Point Dataframe
geometric_points = []
for xy in zip(df_point['Longitude'], df_point['Latitude']):
    geometric_points.append(Point(xy))

pointDF = gpd.GeoDataFrame(df_point,
                                crs = {'init': 'epsg:4326'}, 
                                geometry = geometric_points
                                )
pointDF = pointDF.to_crs(polygonDF.crs)
# Join 2 DFs
joinDF = gpd.sjoin(pointDF,polygonDF, how='inner', predicate='within')
joinDF['polygon'] = joinDF['index_right'].map(polygonDF['geometry'])
print('==Complete Locating point')

### Preprocessing
df_segment = joinDF.copy()
df_segment = df_segment[df_segment['polygon'].notna()]
df_segment = df_segment.dropna(subset='Best Cell Label')
df_segment['polygon_str'] = df_segment['polygon'].astype(str)

***===RUNNING:DN_KCN_HOA_KHANH, 10*10 grid ==***
==Complete dividing bins


c:\Users\MLMT-DATTH-P\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


==Complete Locating point


In [83]:
# Pilot Pollution
pilot_columns = [
    'polygon_str',
    'bin_id',
    'Serving Cell Label',
    'Serving Cell RSRP',
    'Best Cell Label',
    'Best Cell RSRP',
    'Second Best Cell Label',
    'Second Best Cell RSRP',
    'Third Best Cell Label',
    'Third Best Cell RSRP',
    'Fourth Best Cell Label',
    'Fourth Best Cell RSRP',
    'Fifth Best Cell Label',
    'Fifth Best Cell RSRP',
    'Sixth Best Cell Label',
    'Sixth Best Cell RSRP'
]
df_pilot = df_segment[pilot_columns].copy()
df_pilot['rsrp_margin'] = df_pilot['Best Cell RSRP'] - 5
n_bestcell_clumns = [
    'rsrp_margin',
    'Second Best Cell RSRP',
    'Third Best Cell RSRP',
    'Fourth Best Cell RSRP',
    'Fifth Best Cell RSRP',
    'Sixth Best Cell RSRP'
]
compare_column = 'rsrp_margin'
## Code 1
# df_pilot['pilot_polution'] = df_pilot[n_bestcell_clumns].apply(lambda row: sum(row[1:] > row[compare_column]), axis=1)
## Code 2, faster
data_array = df_pilot[n_bestcell_clumns].values
compare_array = df_pilot[compare_column].values
counts = np.sum(data_array[:, 1:] > compare_array[:, np.newaxis], axis=1)
df_pilot['pilot_polution'] = counts
## End of Code 2 ##
df_pilot_agg = df_pilot[['polygon_str','bin_id', 'pilot_polution']].copy()
df_pilot_agg_mean = df_pilot_agg.groupby(['polygon_str','bin_id']).mean()
df_pilot_agg_count = df_pilot_agg.groupby(['polygon_str','bin_id']).count()
df_pilot_agg = df_pilot_agg_mean
df_pilot_agg['No.Samples'] = df_pilot_agg_count['pilot_polution']
df_pilot_agg.reset_index(inplace=True)

In [84]:
from pilot import save_output_to_kml
save_output_to_kml(df_pilot_agg, 'pilot_polution', 'pilot_pollution', 'DN_KCN_HOA_KHANH_PILOT')

## TEST

In [107]:
def save_cluster_outline_kml(boundary_gdf, output):
    kml = Kml()
    for idx, row in boundary_gdf.iterrows():
        polygon = row['geometry']
        # Create a KML polygon with a red outline and no fill color
        kml_polygon = kml.newpolygon(name=row['Name'])
        kml_polygon.outerboundaryis = list(polygon.exterior.coords)
        kml_polygon.description = row['Description']
        color = (255,255,255,128)
        outline_color = ((255, 19, 252, 255))
        kml_polygon.style.linestyle.color = Color.rgb(*outline_color)
        kml_polygon.style.linestyle.width = 2
        kml_polygon.style.polystyle.color = Color.rgb(*color)
    kml.savekmz(f"{output}.kmz")

In [97]:
threshold = 1.33
metric = 'pilot_polution'
eps = 35
min_samples = 5

df = df_pilot_agg[df_pilot_agg['No.Samples'] >= 3].copy()
df['polygon_str'] = df['polygon_str'].str.replace('POLYGON ', '', regex=False).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
df['geometry'] = df['polygon_str'].apply(lambda x: Polygon([tuple(map(float, c.split())) for c in x.split(',')]))
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.crs = "epsg:4326"
gdf = gdf.to_crs('EPSG:32648')
gdf['center_point'] = gdf['geometry'].centroid
gdf['latitude'] = gdf['center_point'].y
gdf['longitude'] = gdf['center_point'].x
bins = [-float('inf'), threshold, float('inf')]
gdf[f"{metric}_cut"] = pd.cut(gdf[metric], bins=bins, labels=False)
if metric == 'Median RSRP':
    gdf = gdf[gdf[f"{metric}_cut"] == 0]
if metric == 'pilot_polution':
    gdf = gdf[gdf[f"{metric}_cut"] == 1]

## DBSCAN
X = gdf[['longitude', 'latitude']]
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(X)

gdf['cluster'] = labels
## Takes instances that belong to a cluster
gdf = gdf[gdf['cluster'] != -1][['geometry','cluster']].to_crs('EPSG:4326')

## Create a Multipolygon for each cluster
grouped_gdf = gdf.groupby('cluster',group_keys=True)['geometry'].apply(lambda x: MultiPolygon(list(x))).reset_index()
multi_polygon_gdf = gpd.GeoDataFrame(grouped_gdf, geometry='geometry', crs=gdf.crs)
multi_polygon_gdf = multi_polygon_gdf.rename(columns={'cluster': 'cluster_id'})
## Create a polygon with the boundary of each multipolygon (each cluster)
boundary_polygons = []
for index, row in multi_polygon_gdf.iterrows():
    geometry = row['geometry']
    boundary = geometry.envelope
    boundary_polygons.append(boundary)
boundary_gdf = gpd.GeoDataFrame({'geometry': boundary_polygons}, crs=multi_polygon_gdf.crs)

In [98]:
intersection_dict = {}
group_id = 0
for idx, polygon in boundary_gdf.iterrows():
    intersects = False
    for key, group in intersection_dict.items():
        if polygon['geometry'].intersects(group):
            intersection_dict[key] = unary_union([group, polygon['geometry']])
            intersects = True
            break
    # If the current polygon doesn't intersect with any existing group, create a new group
    if not intersects:
        intersection_dict[group_id] = polygon['geometry']
        group_id += 1

# Create a GeoDataFrame from the grouped polygons
grouped_gdf = gpd.GeoDataFrame({'geometry': list(intersection_dict.values())}, crs=boundary_gdf.crs)


In [101]:
grouped_gdf.head(3)

,geometry
0,"POLYGON ((108.10908 16.08565, 108.11087 16.085..."
1,"POLYGON ((108.10950 16.08266, 108.11139 16.082..."
2,"POLYGON ((108.11395 16.08052, 108.11442 16.080..."


In [106]:
a = "RSRP" if metric=='Median RSRP' else "Pilot"
def get_name(row):
    return f"{area_name}_{a}_{date_str}_{row.name}"
def get_description(row):
    return f"ID: {area_name}_{a}_{date_str}_{row.name}"
grouped_gdf['Name'] = grouped_gdf.apply(get_name, axis=1)
grouped_gdf['Description'] = grouped_gdf.apply(get_description, axis=1)
grouped_gdf

,geometry,Description,Name
0,"POLYGON ((108.10908 16.08565, 108.11087 16.085...",ID: DN_KCN_HOA_KHANH_Pilot_210723_0,DN_KCN_HOA_KHANH_Pilot_210723_0
1,"POLYGON ((108.10950 16.08266, 108.11139 16.082...",ID: DN_KCN_HOA_KHANH_Pilot_210723_1,DN_KCN_HOA_KHANH_Pilot_210723_1
2,"POLYGON ((108.11395 16.08052, 108.11442 16.080...",ID: DN_KCN_HOA_KHANH_Pilot_210723_2,DN_KCN_HOA_KHANH_Pilot_210723_2
3,"POLYGON ((108.11459 16.08584, 108.11506 16.085...",ID: DN_KCN_HOA_KHANH_Pilot_210723_3,DN_KCN_HOA_KHANH_Pilot_210723_3
4,"POLYGON ((108.11638 16.08023, 108.11442 16.080...",ID: DN_KCN_HOA_KHANH_Pilot_210723_4,DN_KCN_HOA_KHANH_Pilot_210723_4
5,"POLYGON ((108.11714 16.08155, 108.11817 16.081...",ID: DN_KCN_HOA_KHANH_Pilot_210723_5,DN_KCN_HOA_KHANH_Pilot_210723_5
6,"POLYGON ((108.11775 16.08479, 108.11850 16.084...",ID: DN_KCN_HOA_KHANH_Pilot_210723_6,DN_KCN_HOA_KHANH_Pilot_210723_6
7,"POLYGON ((108.11905 16.08442, 108.11943 16.084...",ID: DN_KCN_HOA_KHANH_Pilot_210723_7,DN_KCN_HOA_KHANH_Pilot_210723_7
8,"POLYGON ((108.11907 16.07936, 108.11963 16.079...",ID: DN_KCN_HOA_KHANH_Pilot_210723_8,DN_KCN_HOA_KHANH_Pilot_210723_8
9,"POLYGON ((108.11939 16.08206, 108.12089 16.082...",ID: DN_KCN_HOA_KHANH_Pilot_210723_9,DN_KCN_HOA_KHANH_Pilot_210723_9


In [108]:
save_cluster_outline_kml(grouped_gdf, "test")